In [1]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd 
import numpy as np
import re 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier 

In [3]:
data=pd.read_csv('fake reviews dataset.csv')
data.head(10)

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...
5,Home_and_Kitchen_5,3.0,CG,I WANTED DIFFERENT FLAVORS BUT THEY ARE NOT.
6,Home_and_Kitchen_5,5.0,CG,They are the perfect touch for me and the only...
7,Home_and_Kitchen_5,3.0,CG,These done fit well and look great. I love th...
8,Home_and_Kitchen_5,5.0,CG,"Great big numbers & easy to read, the only thi..."
9,Home_and_Kitchen_5,5.0,CG,My son loves this comforter and it is very wel...


In [4]:
data.isna().sum()

category    0
rating      0
label       0
text_       0
dtype: int64

In [5]:
data.describe()

,rating
count,40432.000000
mean,4.256579
std,1.144354
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [6]:
data['category'].value_counts()

category
Kindle_Store_5                  4730
Books_5                         4370
Pet_Supplies_5                  4254
Home_and_Kitchen_5              4056
Electronics_5                   3988
Sports_and_Outdoors_5           3946
Tools_and_Home_Improvement_5    3858
Clothing_Shoes_and_Jewelry_5    3848
Toys_and_Games_5                3794
Movies_and_TV_5                 3588
Name: count, dtype: int64

In [7]:
data['label'].value_counts()

label
CG    20216
OR    20216
Name: count, dtype: int64

In [8]:
# now changing the label to integers orignal (OR)=1 and computer genterated(CG)=0
data['label']=data['label'].map({'CG':0, "OR":1})
data['label'].value_counts()

label
0    20216
1    20216
Name: count, dtype: int64

In [9]:
data['text_'][0]

'Love this!  Well made, sturdy, and very comfortable.  I love it!Very pretty'

In [10]:
# now working upon the text 
# first make the object of PorterStemmer
stemmers=PorterStemmer()
stop_words_set = stopwords.words('english')
#second take out the special Symbols and  stop words for the text 
def stemming(words):
    words=re.sub('[^a-zA-Z]',' ',words)
    words=words.lower()
    words=words.split()
    words= [stemmers.stem(i) for i in words if i not in stop_words_set ]
    words=' '.join(words)
    return words

In [11]:
try:
    data['text_']=data['category']+data['text_']
    data['text_']=data['text_'].apply(stemming)
    print(data['text_'])
except Exception as e:
    print(e)

0        home kitchen love well made sturdi comfort lov...
1        home kitchen love great upgrad origin mine cou...
2        home kitchen pillow save back love look feel p...
3         home kitchen miss inform use great product price
4         home kitchen nice set good qualiti set two month
                               ...                        
40427    cloth shoe jewelri read review say bra ran sma...
40428    cloth shoe jewelri sure exactli would littl la...
40429    cloth shoe jewelri wear hood wear hood wear ja...
40430    cloth shoe jewelri like noth dress reason gave...
40431    cloth shoe jewelri work wed industri work long...
Name: text_, Length: 40432, dtype: object


In [12]:
X = data.drop('label', axis=1)
y = data['label']

In [13]:
algorithms = {
       
        "SVC": SVC(),
     "Logistic Regression": LogisticRegression(),
        "Random Forest Classifier": RandomForestClassifier(),
        "Decision Tree Classifier": DecisionTreeClassifier(),
        "Gradient Boosting Classifier": GradientBoostingClassifier(),
        "XGBoost Classifier": XGBClassifier()
    }
for i in algorithms.values():
    print(i)

SVC()
LogisticRegression()
RandomForestClassifier()
DecisionTreeClassifier()
GradientBoostingClassifier()
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)


In [14]:
from sklearn.metrics import accuracy_score
X = data['text_']
y = data['label']

X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=0.1, random_state=32)

vector=TfidfVectorizer()
X_train=vector.fit_transform(X_train)
X_test=vector.transform(X_test)

results = {}
for name, model in algorithms.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model: {name}\nTraining score: {model.score(X_train, y_train)}\nTesting score: {accuracy}")

Model: SVC
Training score: 0.9781796196548312
Testing score: 0.8850148367952523
Model: Logistic Regression
Training score: 0.889578982082005
Testing score: 0.8595450049455984
Model: Random Forest Classifier
Training score: 1.0
Testing score: 0.8481701285855588
Model: Decision Tree Classifier
Training score: 1.0
Testing score: 0.7490108803165183
Model: Gradient Boosting Classifier
Training score: 0.8056502143563812
Testing score: 0.7856083086053413
Model: XGBoost Classifier
Training score: 0.8998021325711774
Testing score: 0.8417408506429278
